In [5]:
#| default_exp clean
from hn_jobs_chat.var import postsTableName
from hn_jobs_chat.db import connectToDB


With our new tables created, we now need to create embeddings for each of the items.

First, let's add the embedding column to each of the tables.

In [6]:
conn, cursor = connectToDB()

# TODO: Retreive this from the key model
array_items = "more_company_info","job_requirements","job_soft_skills","tech_stack","information_urls"

for item in array_items:
    tableName = postsTableName + "_" + item

    query = """ALTER TABLE """+tableName+""" ADD COLUMN IF NOT EXISTS embedding float[1536]"""
    cursor.execute(query)
    conn.commit()

conn.close()


Now we can create the embeddings for each of the items.

In [8]:
from openai import OpenAI
client = OpenAI()

conn, cursor = connectToDB()

for item in array_items:
    print('processing field: ' + item)
    tableName = postsTableName + "_" + item

    query = """SELECT * FROM """ + tableName

    cursor.execute(query)

    data = cursor.fetchall()

    for datum in data:
        # if an identical item has already been processed, just use that embedding
        # otherwise, retreive and embedding
        item_id, text = datum[0],datum[2]

        print('processing: ' + str(item_id))

        search_query = """SELECT * FROM """ + tableName + """ WHERE """+item+""" = %s AND embedding IS NOT NULL"""
        cursor.execute(search_query, (text,))
        search_results = cursor.fetchone()

        if search_results is not None:
            embedding = search_results[3]
        else:
            response = client.embeddings.create(
                input=text,
                model="text-embedding-3-small"
            )

            embedding = (response.data[0].embedding)

        query = """UPDATE """ + tableName + """ SET embedding = %s WHERE id = %s"""
        cursor.execute(query, (embedding, item_id))
        conn.commit()
        
conn.close()

NameError: name 'c' is not defined